# Pyagroplan example

---

**Authors:** Margot Challand, Dimitri Justeau-Allaire, Titouan Lorieul, Stéphane de Tourdonnet, Philippe Vismara

**Contact:** philippe.vismara@institut-agro.fr

---

In [ ]:
# Imports general packages needed for this tutorial
from matplotlib import pyplot as plt
from pathlib import Path

# Imports PyAgroPlan
import pyagroplan

## Data loading

In [ ]:
# Defines the data path
DATA_PATH = Path("data/")

# Loads all the data needed for the problem
data = pyagroplan.CropPlanProblemData(
    beds_data=DATA_PATH / "beds_data.csv",
    future_crop_calendar=DATA_PATH / "future_crop_calendar.csv",
    past_crop_plan=DATA_PATH / "past_crop_plan.csv",
    crop_types_attributes=DATA_PATH / "crop_types_attributes.csv",
)

### Beds data

In [ ]:
# Shows the bed data
data.beds_data.df_beds_data.head()

In [ ]:
# Plots the beds adjacency graph

from pyagroplan.plotting import plot_beds_adjacency_graph

fig = plt.figure(figsize=(3, 2))
ax = fig.gca()

ax = plot_beds_adjacency_graph(data.beds_data, "adjacent_beds_in_garden", ax=ax)
ax.set_title("Beds adjacency graph")

### Crop calendar

In [ ]:
# Plots the crop calendar (will be given for each scenario)

from pyagroplan.plotting import plot_crop_calendar

fig = plt.figure(figsize=(8, 4))
ax = fig.gca()

ax = plot_crop_calendar(data.crop_calendar, ax=ax)
ax.set_xlabel("week number")
ax.set_ylabel("need")
ax.set_title("Crops calendar")

fig.tight_layout()

## Model definition and solving

In [ ]:
# Defines the model
model = pyagroplan.AgroEcoPlanModel(data)

# Initializes the model without constraints
model.init()

In [ ]:
# Solves the problem and return a single solution
solution = model.solve(time_limit="30s")

In [ ]:
# Plots the solution (will be given for each scenario)

from pyagroplan.plotting import plot_solution

fig = plt.figure(figsize=(8, 3))
ax = fig.gca()

ax = plot_solution(solution, ax=ax)
ax.set_xlabel("week number")
ax.set_ylabel("beds")
ax.set_title("One possible solution without constraints")

fig.tight_layout()

## Constraints

There are two main types of constraints:
1. Temporal constraints
2. Spatial constraints

### Temporal constraints

#### Return delays

We want to add the following constraint:
> Respect the return delays provided by ITAB (2015).

To do so, we will use a constraint of type `return_delays_constraint` that allows to force return delays between crops based on a matrix containing those return delays.

It can be configured using the following dictionary:

In [ ]:
# Defines the constraint
constraints_definition = {
    "enforce_return_delays": {
        "constraint_type": "return_delays_constraint",
        "return_delays": DATA_PATH / "return_delays.csv",  # Path to the file containing the return delays matrix
    }
}

In [ ]:
# Defines the model
model = pyagroplan.AgroEcoPlanModel(data)

# Initializes the model with the constraint
constraints = pyagroplan.load_constraints(data, constraints_definition)
model.init(constraints)

# Solves the problem and return a single solution satisfying the constraint
solution = model.solve(time_limit="30s")

In [ ]:
# Plots the solution

from pyagroplan.plotting import plot_solution

fig = plt.figure(figsize=(8, 3))
ax = fig.gca()

ax = plot_solution(solution, ax=ax)
ax.set_xlabel("week number")
ax.set_ylabel("beds")
ax.set_title(f"One possible solution with return delays constraint")

fig.tight_layout()

#### Negative precedence effect constraints

We want to add the following constraint:

>Prohibit *preceding crops* that favour weeds before *following crops* that require weed-free soil (because they are sown directly in the field, e.g. carrots) if the *delay* between the two crops is less than 6 weeks.

To do so, we will use a constraint of type `precedence_constraint` that forbids negative precedence effects described in rules defined by the user.

It can be configured using the following dictionary:

In [ ]:
# Defines the constraint
constraints_definition = {
    "forbid_favouring_weed_crop_before_weed-free_soil_requirering_crop": {
        "constraint_type": "precedence_constraint",
        "type": "forbidden",  # Negative precedence constraint
        "precedence_effect_delay_in_weeks": "6",  # Duration of the precedence effect
        "rule": """
            (preceding_crop["effect_on_weeds"] == "favouring")
            & following_crop["requires_weed-free_soil"]
        """,  # Rule selecting the crops on which the constraint applies using fields from crop data
    }
}

In [ ]:
# Defines the model
model = pyagroplan.AgroEcoPlanModel(data)

# Initializes the model with the constraint
constraints = pyagroplan.load_constraints(data, constraints_definition)
model.init(constraints)

# Solves the problem and return a single solution satisfying the constraint
solution = model.solve(time_limit="30s")

In [ ]:
# Plots the solution

from pyagroplan.plotting import plot_solution

fig = plt.figure(figsize=(8, 3))
ax = fig.gca()

ax = plot_solution(solution, ax=ax)
ax.set_xlabel("week number")
ax.set_ylabel("beds")
ax.set_title(f"One possible solution with negative precedence constraint")

fig.tight_layout()

### Spatial constraints

#### Negative spatial interactions constraints

We want to add the following constraint:
> Forbid the assignment of courgettes in adjacent beds to avoid transmission of diseases such as powdery mildew.

To do so, we will use a constraint of type `spatial_interactions_constraint` that forbids negative spatial interactions described in rules defined by the user.

It can be configured using the following dictionary:

In [ ]:
# Defines the constraint
constraints_definition = {
    "dilute_courgettes": {
        "constraint_type": "spatial_interactions_constraint",
        "type": "forbidden",  # Negative spatial interactions constraint
        "adjacency_type": "adjacent_beds_in_garden",  # Adjacency type on which the constraint applies
        "rule": """
            (crop1["crop_type"] == "courgette")
            & (crop2["crop_type"] == "courgette")
        """,  # Rule selecting the crops on which the constraint applies using fields from crop data
        "intervals_overlap": "[1,-1][1,-1]",  # Cultivation intervals during which the constraint applies (can use fields from crop data), here it is the whole cultivation period (i.e., first week to last week)
    }
}

In [ ]:
# Defines the model
model = pyagroplan.AgroEcoPlanModel(data)

# Initializes the model with the constraint
constraints = pyagroplan.load_constraints(data, constraints_definition)
model.init(constraints)

# Solves the problem and return a single solution satisfying the constraint
solution = model.solve(time_limit="30s")

In [ ]:
# Plots the solution

from pyagroplan.plotting import plot_solution

fig = plt.figure(figsize=(8, 3))
ax = fig.gca()

ax = plot_solution(solution, ax=ax)
ax.set_xlabel("week number")
ax.set_ylabel("beds")
ax.set_title(f"One possible solution with negative spatial interactions constraint")

fig.tight_layout()

#### Compatible beds constraints

We want to add the following constraint:
> Forbid beds in the shade to crops requiring exposure to the sun (in our case, vegetable fruits such as courgettes, eggplants, etc.).

To do so, we will use `compatible_beds_constraint` that allows to forbid or enforce certain combinations of beds and crops.

It can be configured using the following dictionary:

In [ ]:
# Defines the constraint
constraints = {
    "avoid_growing_crops_requiring_sun_exposure_to_shady_beds": {
        "constraint_type": "compatible_beds_constraint",
        "type": "forbidden",  # Incompatible beds constraint
        "crops_selection_rule": """
            crop["requires_sun_exposure"] == "full_sun"
        """,  # Rule selecting the crops on which the constraint applies using fields from crop data)
        "beds_selection_rule": """
            bed["in_shade"]
        """,  # Rule selecting the beds on which are enforced or forbidden using fields from bed data (can also using fields from crop data)
    }
}

compatible_beds_constraints = pyagroplan.load_constraints(data, constraints)

In [ ]:
# Defines the model
model = pyagroplan.AgroEcoPlanModel(data)

# Initializes the model with the constraint
constraints = pyagroplan.load_constraints(data, constraints_definition)
model.init(constraints)

# Solves the problem and return a single solution satisfying the constraint
solution = model.solve(time_limit="30s")

In [ ]:
# Plots the solution

from pyagroplan.plotting import plot_solution

fig = plt.figure(figsize=(8, 3))
ax = fig.gca()

ax = plot_solution(solution, ax=ax)
ax.set_xlabel("week number")
ax.set_ylabel("beds")
ax.set_title(f"One possible solution with compatible beds constraint")

fig.tight_layout()

#### Group crops

We want to add the following constraint:
> Regroup eggplants to ease their management due to operational considerations.

To do so, we will use `group_crops_constraint` that spatially regroups crops chosen by the user.

It can be configured using the following dictionary:

In [ ]:
constraints_definition = {
    "group_crops": {
        "constraint_type": "group_crops_constraint",
        "adjacency_type": "adjacent_beds_in_garden",  # Adjacency type on which the constraint applies
        "group_by": "crop_group_id",  # Field name on which the groups will be created, here groups by crops defined on the same line of future_crop_calendar.csv
        "filtering_rule": """
            (crop["crop_type"] == "eggplant")
        """,  # Rule selecting the crops on which the constraint applies using fields from crop data, here keeps only crops with a crop type equal to "eggplant"
    },
}

In [ ]:
# Defines the model
model = pyagroplan.AgroEcoPlanModel(data)

# Initializes the model with the constraint
constraints = pyagroplan.load_constraints(data, constraints_definition)
model.init(constraints)

# Solves the problem and return a single solution satisfying the constraint
solution = model.solve(time_limit="30s")

In [ ]:
from pyagroplan.plotting import plot_solution

fig = plt.figure(figsize=(8, 3))
ax = fig.gca()

ax = plot_solution(solution, ax=ax)
ax.set_xlabel("week number")
ax.set_ylabel("beds")
ax.set_title(f"One possible solution with group crops constraint")

fig.tight_layout()

## Final model with all constraints

In [ ]:
# Defines the constraints
constraints_definition = {
    "enforce_return_delays": {
        "constraint_type": "return_delays_constraint",
        "return_delays": DATA_PATH / "return_delays.csv",
    },
    "forbid_favouring_weed_crop_before_weed-free_soil_requirering_crop": {
        "constraint_type": "precedence_constraint",
        "type": "forbidden",
        "precedence_effect_delay_in_weeks": "6",
        "rule": """
            (preceding_crop["effect_on_weeds"] == "favouring") & following_crop["requires_weed-free_soil"]
        """,
    },
    "dilute_courgettes": {
        "constraint_type": "spatial_interactions_constraint",
        "type": "forbidden",
        "adjacency_type": "adjacent_beds_in_garden",
        "rule": """
            (crop1["crop_type"] == "courgette")
            & (crop2["crop_type"] == "courgette")
        """,
        "intervals_overlap": "[1,-1][1,-1]",
    },
    "avoid_growing_crops_requiring_sun_exposure_to_shady_beds": {
        "constraint_type": "compatible_beds_constraint",
        "type": "forbidden",
        "crops_selection_rule": """
            crop["requires_sun_exposure"] == "full_sun"
        """,
        "beds_selection_rule": """
            bed["in_shade"]
        """,
    },
    "group_crops": {
        "constraint_type": "group_crops_constraint",
        "adjacency_type": "adjacent_beds_in_garden",
        "group_by": "crop_group_id",
        "filtering_rule": """
            (crop["crop_type"] == "eggplant")
        """,
    },
}

# Defines the model
model = pyagroplan.AgroEcoPlanModel(data)

# Initializes the model with all the constraints
constraints = pyagroplan.load_constraints(data, constraints_definition)
model.init(constraints)

# Solves the problem and return a single solution satisfying the constraint
solution = model.solve(time_limit="30s")

In [ ]:
# Prints some statistics about the constraints
model.print_constraints_statistics()

In [ ]:
# Plots the solution

from pyagroplan.plotting import plot_solution

fig = plt.figure(figsize=(8, 3))
ax = fig.gca()

ax = plot_solution(solution, ax=ax)
ax.set_xlabel("week number")
ax.set_ylabel("beds")
ax.set_title(f"One possible solution with all constraints")

fig.tight_layout()